# convolution

convolution is one of the easiest - yet most obfuscated mathematical operations. 

**discrete convolution is a weighted overlap and add operation. **

*that's it.*

I cannot tell you how many signal processing texts I have read that go into long winded missives about the complexities of convolution. Yes, there are complexities - but there are also some really simple things about convolution that are should not be lost at the expense of the complicated things.

ok - here we go. Let's start simple and see how convolution works through examples. 

*(these examples look at convolution in the context of numpy)*

In [1]:
import numpy as np

### phase 1 - so easy you wonder why we're doing this

Convolution will produce an output sequence from the inputs via numerical magic. The pattern by which the magic plays can be seen if you:

* Think of `x` as a set of weights for each index
* Think of `y` as an entity to scaled and added to the output by the weighted prescriptions from `x`. 

So, to generate an output with the number 99 scaled by factor of `1` at the second location, you would say

In [2]:
x = [0, 1, 0, 0]
y = [99]

z = np.convolve(x, y, 'full')
print(z)

[ 0 99  0  0]


So what did this do? It generated an output sequence in which the number `99` was scaled by:
* `0` at the first location
* `1` at the second location
* `0` at the third location
* `0` at the fourth location

To generate an output sequence with the number 99 scaled by a factor of `1` at both the second and fourth locations, you would say

In [3]:
x = [0, 1, 0, 1]
y = [99]

z = np.convolve(x, y, 'full')
print(z)

[ 0 99  0 99]


To create an output sequence with the number 99 scaled by
* a factor of `1` at the second location _and_
* a factor of `2` at the fourth location _and_
* a factor of `0` at all other locations

you would say

In [4]:
x = [0, 1, 0, 2]
y = [99]

z = np.convolve(x, y, 'full')
print(z)

[  0  99   0 198]


which is consistent with what was observed in the previous example.

Something to note at this point is that `0` is different from 'not there.' The above output is length `4` because we stuffed zeros in at the first and third locations. The output would be shorter if we left them out - which can be seen in the following operations which produce very different, but predictable output sequences.

In [5]:
x = [2, 4]
y = [10]

z = np.convolve(x, y, 'full')
print(z, "\n")

x = [2, 0, 0, 4]
y = [10]

z = np.convolve(x, y, 'full')
print(z, "\n")

x = [2, 1, 1, 4]
y = [10]

z = np.convolve(x, y, 'full')
print(z, "\n")

[20 40] 

[20  0  0 40] 

[20 10 10 40] 



### phase 2 - should make sense if you've read phase 1

At this point, we're going to change things up a _little_ bit. Our weighted output index instruction vector is going to be bascially the same, but the entity we scale, `y`, is going to be a vector. Again, let's make the following assumptions:

* Thinking of `x` as a set of weights for each index of the output
* Think of `y` as an entity to scaled and added to the output by the weighted prescriptions from `x`. 

The first operation will be to scale the vector `[10, 10]` with a factor of `5` and overlap it with the output at the first location

In [6]:
weights_by_index = [5]
target_to_scale  = [10, 10]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z)

[50 50]


Note that the `weights_by_index` vector only has a length of `1`, but the `target_to_scale` vector has a length of `2`. This means a couple things: 

* the output we create are going to have to have a length of `2` (since we're not going to drop any of the scaled computations we have performed)
* that the first and second slots of the output will be filled with `target_to_scale.` weighted by a factor of `5`.

It is important to observe that the output winds up being longer than the `weights_by_index` vector. Let's a look at another example:

In [7]:
weights_by_index = [5]
target_to_scale  = [1, 2, 3, 4, 5]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z)

[ 5 10 15 20 25]


The length of this output should make sense. `target_to_scale` was scaled by a factor of five and inserted in the first location.

In the following example, we'll have a longer `weights_by_index` vector. The target will be scaled and applied to the output at the first and fifth locations as prescibed

In [8]:
weights_by_index = [2, 0, 0, 0, 1]
target_to_scale  = [2, 4, 6]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z)

[ 4  8 12  0  2  4  6]


Here's another example. Note that the length of the output sequence is *always* `length(x) + length(y) - 1` (verify this with the other examples on the page)

In [9]:
weights_by_index = [2, 0, 1, 0, 0.5]
target_to_scale  = [2, 6]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z)

[  4.  12.   2.   6.   1.   3.]


### a little secret about convolution

Guess what? You may not have noticed this, but the order of `weights_by_index` and `target_to_scale` doesn't matter. Convolution is a linear operator, and as such has the commutative property (as well as associative and distributive properties).

Not convinced? Examine this example with the "weights" and "targets" logic in mind.

In [10]:
weights_by_index = [2, 0, 1]
target_to_scale  = [4]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

weights_by_index = [4]
target_to_scale  = [2, 0, 1]


z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

[8 0 4] 

[8 0 4] 



or the following example 

In [11]:
weights_by_index = [1, 0, 0]
target_to_scale  = [0, 1]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

weights_by_index = [0, 1]
target_to_scale  = [1, 0, 0]


z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

[0 1 0 0] 

[0 1 0 0] 



note 

### phase 3 - starting to get a little convoluted

Let's still operate from a mindset of `weights_by_index` and `target_to_scale` for purposes of organization in your brain parts. 

Now I'm going to do something fancy here, I'm going to make `target_to_scale` a vector that is long enough that it can _overlap_ a previous application from `weights_by_index`. Let's step through from non-overlapped to overlapped...

In [12]:
weights_by_index = [1, 0, 0, 0, 10]
target_to_scale  = [3, 4]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

weights_by_index = [1, 0, 0, 10]
target_to_scale  = [3, 4]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

weights_by_index = [1, 0, 10]
target_to_scale  = [3, 4]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

weights_by_index = [1, 10]
target_to_scale  = [3, 4]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

[ 3  4  0  0 30 40] 

[ 3  4  0 30 40] 

[ 3  4 30 40] 

[ 3 34 40] 



Ok, did you see that? The `30` and the `4` overlapped to create `34`! That was pretty awesome. 

If that result makes sense, then numerically you get convolution. We can go one step further and see how a single cell can be the application of weighted add operations. Here, the third cell is the sum of a weight 1 operation, a weight 10 operation and a weight 100 operation

In [13]:
weights_by_index = [1, 1, 1]
target_to_scale  = [1, 10, 100]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

[  1  11 111 110 100] 



Below is more of a standard teaching example. I kind of hate this this example, but I feel that above descrtiption should make the output of the operation a lot more intuitive to you than it was five minutes ago:

In [14]:
weights_by_index = [1, 1, 1]
target_to_scale  = [1, 1, 1]

z = np.convolve(weights_by_index, target_to_scale, 'full')
print(z, "\n")

[1 2 3 2 1] 

